## Init

In [ ]:
!pip install datasets transformers pandas tqdm matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
squad_v2 = True

In [ ]:
def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    return model, tokenizer

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from tqdm.auto import tqdm

def evaluate_model(model, tokenizer, dataset, subset_size=100):
    nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

    # Determine if 'validation' split is available, otherwise use 'train'
    # split_to_use = 'validation' if 'validation' in dataset else 'train'
    #split_to_use = 'test'
    split_to_use = 'train'

    # Select a subset from the appropriate split
    subset = dataset[split_to_use].select(range(subset_size))
    metric = load_metric("squad_v2")
    results = []

    for example in tqdm(subset, desc=f"Evaluating ({split_to_use} split)", unit="example"):
        outputs = nlp({
            "question": example["question"],
            "context": example["context"]
        })
        formatted_predictions = {
            "prediction_text": outputs['answer'],
            "id": example["id"],
            "no_answer_probability": 0.0
        }
        formatted_references = {
            "answers": example["answers"],
            "id": example["id"]
        }
        metric.add(prediction=formatted_predictions, reference=formatted_references)

    final_scores = metric.compute()
    return final_scores

In [ ]:
# Initialize an empty list to hold the datasets
all_datasets = [
    # ("squad_v2" if squad_v2 else "squad", load_dataset("squad_v2" if squad_v2 else "squad")),

    # ("cuad", load_dataset("cuad")),

    # ("covid_qa_deepset", load_dataset("covid_qa_deepset")),

    ("TriviaQA_SQuAD", load_dataset("Kkordik/TriviaQA_SQuAD")),

    # ("books", load_dataset("subjqa", "books")),
    # ("movies", load_dataset("subjqa", "movies")),
    # ("restaurants", load_dataset("subjqa", "restaurants")),
    # ("electronics", load_dataset("subjqa", "electronics")),
    # ("grocery", load_dataset("subjqa", "grocery")),
    # ("tripadvisor", load_dataset("subjqa", "tripadvisor"))
    # Add other datasets as needed
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14229 [00:00<?, ? examples/s]

In [ ]:
# Liste des modèles à évaluer
models_checkpoint = [
    "ALOQAS/squeezebert-uncased-finetuned-squad-v2",
    "ALOQAS/bert-large-uncased-finetuned-squad-v2",
    "ALOQAS/deberta-large-finetuned-squad-v2"
]

## Squad V2

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (validation split):   0%|          | 0/11873 [00:00<?, ?example/s]

Metric                                                  HasAns_exact  \
Dataset  Model                                                         
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2      81.443995   
         ALOQAS/deberta-large-finetuned-squad-v2           88.090418   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2     72.216599   

Metric                                                  HasAns_f1  \
Dataset  Model                                                      
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2   89.249316   
         ALOQAS/deberta-large-finetuned-squad-v2        94.509947   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2  81.731813   

Metric                                                  HasAns_total  \
Dataset  Model                                                         
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2         5928.0   
         ALOQAS/deberta-large-finetuned-squad-v2              5928.0   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2        5928.0   

Metric                                                  NoAns_exact  NoAns_f1  \
Dataset  Model                                                                  
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2      0.000000  0.000000   
         ALOQAS/deberta-large-finetuned-squad-v2           0.016821  0.016821   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2     0.000000  0.000000   

Metric                                                  NoAns_total  \
Dataset  Model                                                        
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2        5945.0   
         ALOQAS/deberta-large-finetuned-squad-v2             5945.0   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2       5945.0   

Metric                                                  best_exact  \
Dataset  Model                                                       
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2    50.113703   
         ALOQAS/deberta-large-finetuned-squad-v2         50.113703   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2   50.113703   

Metric                                                  best_exact_thresh  \
Dataset  Model                                                              
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2                 0.0   
         ALOQAS/deberta-large-finetuned-squad-v2                      0.0   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2                0.0   

Metric                                                    best_f1  \
Dataset  Model                                                      
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2   50.113703   
         ALOQAS/deberta-large-finetuned-squad-v2        50.748121   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2  50.113703   

Metric                                                  best_f1_thresh  \
Dataset  Model                                                           
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2              0.0   
         ALOQAS/deberta-large-finetuned-squad-v2                   0.0   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2             0.0   

Metric                                                      exact         f1  \
Dataset  Model                                                                 
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2   40.663691  44.560763   
         ALOQAS/deberta-large-finetuned-squad-v2        43.990567  47.195736   
         ALOQAS/squeezebert-uncased-finetuned-squad-v2  36.056599  40.807394   

Metric                                                    total  
Dataset  Model                                                   
squad_v2 ALOQAS/bert-large-uncased-finetuned-squad-v2   11873.0  
         ALOQAS/deberta-large-finetuned-squad-v2        11873.0  
         ALOQAS/squeezebert-uncased-finetuned-squad

## Subjqa

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/512 [00:00<?, ?example/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/512 [00:00<?, ?example/s]

Metric                                                     HasAns_exact  \
Dataset     Model                                                         
books       ALOQAS/bert-large-uncased-finetuned-squad-v2       4.854369   
            ALOQAS/deberta-large-finetuned-squad-v2            6.796117   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      1.941748   
electronics ALOQAS/bert-large-uncased-finetuned-squad-v2       3.750000   
            ALOQAS/deberta-large-finetuned-squad-v2            6.875000   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      1.875000   
grocery     ALOQAS/bert-large-uncased-finetuned-squad-v2       4.477612   
            ALOQAS/deberta-large-finetuned-squad-v2            6.716418   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      3.731343   
movies      ALOQAS/bert-large-uncased-finetuned-squad-v2       1.162791   
            ALOQAS/deberta-large-finetuned-squad-v2            8.139535   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      2.325581   
restaurants ALOQAS/bert-large-uncased-finetuned-squad-v2       6.896552   
            ALOQAS/deberta-large-finetuned-squad-v2            5.747126   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      5.747126   
tripadvisor ALOQAS/bert-large-uncased-finetuned-squad-v2       8.135593   
            ALOQAS/deberta-large-finetuned-squad-v2           10.847458   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2      5.762712   

Metric                                                     HasAns_f1  \
Dataset     Model                                                      
books       ALOQAS/bert-large-uncased-finetuned-squad-v2   26.878272   
            ALOQAS/deberta-large-finetuned-squad-v2        28.863716   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  27.121202   
electronics ALOQAS/bert-large-uncased-finetuned-squad-v2   27.806655   
            ALOQAS/deberta-large-finetuned-squad-v2        30.080849   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  24.848015   
grocery     ALOQAS/bert-large-uncased-finetuned-squad-v2   26.792789   
            ALOQAS/deberta-large-finetuned-squad-v2        30.647142   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  30.195135   
movies      ALOQAS/bert-large-uncased-finetuned-squad-v2   24.241630   
            ALOQAS/deberta-large-finetuned-squad-v2        34.440390   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  22.469376   
restaurants ALOQAS/bert-large-uncased-finetuned-squad-v2   33.606390   
            ALOQAS/deberta-large-finetuned-squad-v2        28.566750   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  32.743289   
tripadvisor ALOQAS/bert-large-uncased-finetuned-squad-v2   40.891465   
            ALOQAS/deberta-large-finetuned-squad-v2        45.571109   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2  36.891595   

Metric                                                     HasAns_total  \
Dataset     Model                                                         
books       ALOQAS/bert-large-uncased-finetuned-squad-v2          103.0   
            ALOQAS/deberta-large-finetuned-squad-v2               103.0   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2         103.0   
electronics ALOQAS/bert-large-uncased-finetuned-squad-v2          160.0   
            ALOQAS/deberta-large-finetuned-squad-v2               160.0   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2         160.0   
grocery     ALOQAS/bert-large-uncased-finetuned-squad-v2          268.0   
            ALOQAS/deberta-large-finetuned-squad-v2               268.0   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2         268.0   
movies      ALOQAS/bert-large-uncased-finetuned-squad-v2           86.0   
            ALOQAS/deberta-large-finetuned-squad-v2                86.0   
            ALOQAS/squeezebert-uncased-finetuned-squad-v2          86.0   
restaura

## cuad

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/100 [00:00<?, ?example/s]

Metric                                                 HasAns_exact  \
Dataset Model                                                         
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2       7.142857   
        ALOQAS/deberta-large-finetuned-squad-v2            0.000000   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2      0.000000   

Metric                                                 HasAns_f1  \
Dataset Model                                                      
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2   20.002342   
        ALOQAS/deberta-large-finetuned-squad-v2         9.263134   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2   9.452988   

Metric                                                 HasAns_total  \
Dataset Model                                                         
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2           28.0   
        ALOQAS/deberta-large-finetuned-squad-v2                28.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2          28.0   

Metric                                                 NoAns_exact  NoAns_f1  \
Dataset Model                                                                  
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2           0.0       0.0   
        ALOQAS/deberta-large-finetuned-squad-v2                0.0       0.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2          0.0       0.0   

Metric                                                 NoAns_total  \
Dataset Model                                                        
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2          72.0   
        ALOQAS/deberta-large-finetuned-squad-v2               72.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2         72.0   

Metric                                                 best_exact  \
Dataset Model                                                       
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2         72.0   
        ALOQAS/deberta-large-finetuned-squad-v2              72.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2        72.0   

Metric                                                 best_exact_thresh  \
Dataset Model                                                              
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2                 0.0   
        ALOQAS/deberta-large-finetuned-squad-v2                      0.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2                0.0   

Metric                                                 best_f1  \
Dataset Model                                                    
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2      72.0   
        ALOQAS/deberta-large-finetuned-squad-v2           72.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2     72.0   

Metric                                                 best_f1_thresh  exact  \
Dataset Model                                                                  
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2              0.0    2.0   
        ALOQAS/deberta-large-finetuned-squad-v2                   0.0    0.0   
        ALOQAS/squeezebert-uncased-finetuned-squad-v2             0.0    0.0   

Metric                                                       f1  total  
Dataset Model                                                           
cuad    ALOQAS/bert-large-uncased-finetuned-squad-v2   5.600656  100.0  
        ALOQAS/deberta-large-finetuned-squad-v2        2.593678  100.0  
        ALOQAS/squeezebert-uncased-finetuned-squad-v2  2.646837  100.0

## covid_qa_deepset

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (train split):   0%|          | 0/250 [00:00<?, ?example/s]

Metric                                                          HasAns_exact  \
Dataset          Model                                                         
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2           27.2   
                 ALOQAS/deberta-large-finetuned-squad-v2                36.8   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2          22.0   

Metric                                                          HasAns_f1  \
Dataset          Model                                                      
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2   42.795043   
                 ALOQAS/deberta-large-finetuned-squad-v2        54.157657   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2  36.319472   

Metric                                                          HasAns_total  \
Dataset          Model                                                         
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2          250.0   
                 ALOQAS/deberta-large-finetuned-squad-v2               250.0   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2         250.0   

Metric                                                          best_exact  \
Dataset          Model                                                       
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2         27.2   
                 ALOQAS/deberta-large-finetuned-squad-v2              36.8   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2        22.0   

Metric                                                          best_exact_thresh  \
Dataset          Model                                                              
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2                 0.0   
                 ALOQAS/deberta-large-finetuned-squad-v2                      0.0   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2                0.0   

Metric                                                            best_f1  \
Dataset          Model                                                      
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2   42.795043   
                 ALOQAS/deberta-large-finetuned-squad-v2        54.157657   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2  36.319472   

Metric                                                          best_f1_thresh  \
Dataset          Model                                                           
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2              0.0   
                 ALOQAS/deberta-large-finetuned-squad-v2                   0.0   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2             0.0   

Metric                                                          exact  \
Dataset          Model                                                  
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2    27.2   
                 ALOQAS/deberta-large-finetuned-squad-v2         36.8   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2   22.0   

Metric                                                                 f1  \
Dataset          Model                                                      
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2   42.795043   
                 ALOQAS/deberta-large-finetuned-squad-v2        54.157657   
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2  36.319472   

Metric                                                          total  
Dataset          Model                                                 
covid_qa_deepset ALOQAS/bert-large-uncased-finetuned-squad-v2   250.0  
                 ALOQAS/deberta-large-finetuned-squad-v2        250.0  
                 ALOQAS/squeezebert-uncased-finetuned-squad-v2  250.0

## Kkordik/TriviaQA_SQuAD

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (train split):   0%|          | 0/100 [00:00<?, ?example/s]

Metric                                                        HasAns_exact  \
Dataset        Model                                                         
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2      34.117647   
               ALOQAS/deberta-large-finetuned-squad-v2           31.764706   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2     25.882353   

Metric                                                        HasAns_f1  \
Dataset        Model                                                      
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2   40.196078   
               ALOQAS/deberta-large-finetuned-squad-v2        42.509804   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2  33.098039   

Metric                                                        HasAns_total  \
Dataset        Model                                                         
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2           85.0   
               ALOQAS/deberta-large-finetuned-squad-v2                85.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2          85.0   

Metric                                                        NoAns_exact  \
Dataset        Model                                                        
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2           0.0   
               ALOQAS/deberta-large-finetuned-squad-v2                0.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2          0.0   

Metric                                                        NoAns_f1  \
Dataset        Model                                                     
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2        0.0   
               ALOQAS/deberta-large-finetuned-squad-v2             0.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2       0.0   

Metric                                                        NoAns_total  \
Dataset        Model                                                        
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2          15.0   
               ALOQAS/deberta-large-finetuned-squad-v2               15.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2         15.0   

Metric                                                        best_exact  \
Dataset        Model                                                       
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2         29.0   
               ALOQAS/deberta-large-finetuned-squad-v2              28.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2        24.0   

Metric                                                        best_exact_thresh  \
Dataset        Model                                                              
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2                 0.0   
               ALOQAS/deberta-large-finetuned-squad-v2                      0.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2                0.0   

Metric                                                          best_f1  \
Dataset        Model                                                      
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2   34.166667   
               ALOQAS/deberta-large-finetuned-squad-v2        36.466667   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2  29.133333   

Metric                                                        best_f1_thresh  \
Dataset        Model                                                           
TriviaQA_SQuAD ALOQAS/bert-large-uncased-finetuned-squad-v2              0.0   
               ALOQAS/deberta-large-finetuned-squad-v2                   0.0   
               ALOQAS/squeezebert-uncased-finetuned-squad-v2             0.0   

Metric                                                        exact  \
Dataset        Model                                  